<a href="https://colab.research.google.com/github/aliakbarmdzh/Loan_prediction/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing libraries

In [ ]:
#General
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#For ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

Loading data

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ProsperLoanData/prosper_cleaned.csv")

Looking at columns

In [ ]:
data.columns

Defining x and y, dropping some of unrelevant features from x

In [ ]:
x=data.drop(['Unnamed: 0','LoanStatus','ListingCategory (numeric)','BorrowerState','DelinquentBorrowers',
             'ListingCreationDate'],axis=1)
y=data['DelinquentBorrowers']

In [ ]:
x.head()

finding high correlated features

In [ ]:
corr=x.corr()
f, ax = plt.subplots(figsize=(11, 9))
mask = np.triu(np.ones_like(corr, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, cmap=cmap, mask=mask,vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

dropping high correlated features

In [ ]:
x=x.drop(['CreditScoreRangeUpper', 'TotalInquiries', 'OpenRevolvingAccounts','MonthlyLoanPayment',
          'OpenRevolvingMonthlyPayment'],axis=1)

Changing EmploymentStatus feature to dummy variables

In [ ]:
set(x.EmploymentStatus)

In [ ]:
x['Employment_Status_Employed']=0
for i in range(len(x['Employment_Status_Employed'])):
  if (x['EmploymentStatus'][i] in ['Self-employed','Full-time','Employed']):
    x['Employment_Status_Employed'][i]=1
x['Employment_Status_Not_Employed']=0
for i in range(len(x['Employment_Status_Not_Employed'])):
  if (x['EmploymentStatus'][i]=='Not employed'):
    x['Employment_Status_Not_Employed'][i]=1
x['Employment_Status_Part_time']=0
for i in range(len(x['Employment_Status_Part_time'])):
  if (x['EmploymentStatus'][i]=='Part-time'):
    x['Employment_Status_Part_time'][i]=1
x.drop('EmploymentStatus',axis=1,inplace=True)

Checking datatypes

In [ ]:
x.dtypes

Definig scoring

In [ ]:
scoring=['precision','accuracy','recall']
def print_scores(scores):
    print("Average accuracy:")
    print(scores['test_accuracy'].mean())
    print("Average precision:")
    print(scores['test_precision'].mean())
    print("Average recall:")
    print(scores['test_recall'].mean())
    return

Modeling

In [ ]:
#Decision Tree
model=DecisionTreeClassifier(random_state=0)

scores_dt=cross_validate(model,x,y,cv=5,scoring=scoring)
print_scores(scores_dt)

In [ ]:
#Random Forest
model=RandomForestClassifier(n_estimators=100,random_state=0)

scores_rf=cross_validate(model,x,y,cv=5,scoring=scoring)
print_scores(scores_rf)

In [ ]:
#XGBClassifier
model = XGBClassifier(n_estimators=100,learning_rate=0.1,n_jobs=4,
                      use_label_encoder=False,eval_metric='logloss',scale_pos_weight=2.2)

scores_xgb=cross_validate(model,x,y,cv=5,scoring=scoring)
print_scores(scores_xgb)